## SPUR Densification Survey- Natural Langugage Processing Project
notebook author: Emily Robitschek

#### Background information on survey: 
This survey was conducted by the SPUR group at ETH: https://spur.ethz.ch/

#### Hypothesis/Research question: 
Can we use the open text field responses from densification surveys to infer something about the motivations behind why individuals responded the way they did to the densification survey?

## Stage 3: Topic modelling of the pre-processed data with LDA
To move beyond top individual words and better characterize themes themes or “topics” in the responses that don’t rely as much on a pre-stratification of the data along a particular response category, I carried out further preprocessing and topic modelling on the text data. Topic modeling is a method for unsupervised classification of documents, similar to clustering on numeric data, which finds some natural groups of items (topics). Topic modeling techniques (specifically Latent Dirichlet Allocation or LDA) have been successfully applied in the context of NLP and text data to extract the main themes from the document collection without having to define them based on a priori expectations, although such methods have their limitations.

In [ ]:
## import necessary packages: 
import pandas as pd
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import seaborn as sns
from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words (extend if desired)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['proposal', 'proposals'])

In [ ]:
## Helper functions

#from this post: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#18dominanttopicineachsentence

# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

        
def get_perplexity_coherence(lda_model, corpus, data_lemmatized, id2word, coherence='c_v'): 
    """Return perplexity and coherence for lda model."""
    # Compute Perplexity
    perplexity = lda_model.log_perplexity(corpus)
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return perplexity, coherence_lda
    

def score_lda_model(min_topic_num, max_topic_num, lda_model, corpus, data_lemmatized, id2word):
    """Return perplexity and coherence for different number of topics for lda model."""
    perplexities = []
    coherences = []
    topic_nums_to_try = list(range(min_topic_num, max_topic_num))
    for num in topic_nums_to_try: 
        print('Running LDA with %d topics' % (num))
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        perplexity, coherence = get_perplexity_coherence(lda_model, corpus, data_lemmatized, id2word, coherence='c_v')
        perplexities.append(perplexity)
        coherences.append(coherence)
    lda_model_eval_df = pd.DataFrame({'number_of_topics': topic_nums_to_try, 
                                 'perplexity': perplexities, 
                                 'coherence': coherences})
    return lda_model_eval_df



### Load in data

In [ ]:
subset_col = 'Q12.6_corrected' #specify corrected text column
survey_df = pd.read_csv('../../datasets/spur_survey_response_filtered_df1.txt', sep='\t')
survey_df.head()

### Preprocess data for model

In [ ]:
#specify text data
data = survey_df[subset_col]
pprint(data[:1])

data_words = list(sent_to_words(data))
print(data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=80) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:5])

In [ ]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus)

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

### Build model (LDA) and assess performance across K

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model_eval_df = score_lda_model(5, 14, lda_model, corpus, data_lemmatized, id2word)
lda_model_eval_df

In [ ]:
#Plot perplexity across K
sns.lineplot(data=lda_model_eval_df, x="number_of_topics", y="perplexity", 
             color='red', linewidth=2.5)
title = ("Perplexity scores for LDA models with different numbers of predicted topics")
plt.xticks(lda_model_eval_df["number_of_topics"])
plt.title(title)
plt.show()


#Plot coherence across K
sns.lineplot(data=lda_model_eval_df, x="number_of_topics", y="coherence",
            color='blue', linewidth=2.5)
title = ("Coherence scores for LDA models with different numbers of predicted topics")
plt.xticks(lda_model_eval_df["number_of_topics"])
plt.title(title)
plt.show()

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=11, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 11 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
# Visualize the topics
vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, sort_topics=False)
print(vis_data.topic_order)
vis_data

### Build model (LDAMallet) and assess performance across K
https://stackoverflow.com/questions/62581874/gensim-ldamallet-vs-ldamodel

In [ ]:
mallet_path = '../../resources/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
#using this function that can take a long time to run from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ 
#because mine includes perplexity and that gave me some initial issues

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, 
                                                        texts=data_lemmatized, start=5, limit=14, step=1)

In [ ]:
# Show graph
limit=14; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Select the model and print the topics
optimal_model = ldamallet 
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=15))

### Relate the topics back to the individual responses

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
for i in range(0,12):
    print("Topic_", i)
    print(sent_topics_sorteddf_mallet['Keywords'][i])
    print(sent_topics_sorteddf_mallet['Text'][i])
    print( )
sent_topics_sorteddf_mallet

In [ ]:
for i in range(0,12):
    print("Topic_", i)
    print("Top 10 Keywords:", sent_topics_sorteddf_mallet['Keywords'][i])
    print("EXAMPLE: ", "'", sent_topics_sorteddf_mallet['Text'][i], "'")
    print( )
sent_topics_sorteddf_mallet

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head(12) #.head(25)

In [ ]:
df_dominant_topics['Dominant_Topic'].value_counts()

### Merge topic data and survey data to be able to look at dominant topic prevalance compared to other survey responses

In [ ]:
survey_df_w_lda = survey_df.merge(df_dominant_topic, left_index=True, right_index=True)
survey_df_w_lda

In [ ]:
survey_df_w_lda[df_dominant_topic['Dominant_Topic']==3.0]['Text']

### Climate subgroups

In [ ]:
# Groupby by climate
climate = survey_df_w_lda.groupby('Q14.9')

plt.figure(figsize=(4,4))
climate.size().sort_values(ascending=False).plot.bar()
plt.xticks(rotation=50)
plt.xlabel("Climate concern")
plt.ylabel("Response count")
plt.show()

# Summary statistic of all countries
climate['Dominant_Topic'].describe().head()

In [ ]:
scale_mapper = {'Not concerned': -2, 'Rather not concerned': -1, 'Neither': 0, 'Quite concerned': 1, 'Very concerned': 2}
survey_df_w_lda["level_climate_concern"] = survey_df_w_lda["Q14.9"].replace(scale_mapper)
survey_df_w_lda

summary_df = survey_df_w_lda[["level_climate_concern", 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

In [ ]:
cross_tab_prop = pd.crosstab(index=summary_df['level_climate_concern'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")
climate_v_lda = cross_tab_prop
climate_v_lda['level_climate_concern'] = climate_v_lda.index.copy()
#cross_tab_prop = cross_tab_prop.reindex(index = ['Not concerned', 'Rather not concerned', 'Neither', 'Quite concerned', 'Very concerned'])
cross_tab_prop #cross_tab_prop.sort_values('very-not concerned', ascending=False) #.corr()
climate_v_lda.corr()
df_corr = climate_v_lda.corr()

In [ ]:
#make heatmap of plot above
plt.figure(figsize=(12, 12))
corr = df_corr.corr(method='spearman') #[['A']]
ax = sns.clustermap(corr, #corr[["level_climate_concern"]],
                 robust=True, 
                 annot=True,  
                 cmap="YlGnBu")

title = ("Correlation heatmap of climate concern and dominant LDA topic" )
plt.title(title)
file_name=('../../figures/' + title + ('_v1_spearman.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')

In [ ]:
summary_df = survey_df_w_lda[['Q14.9', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['Dominant_Topic'],
                             columns=summary_df['Q14.9'],
                             normalize="index")

cross_tab_prop['very-not concerned'] =  cross_tab_prop['Very concerned'] - cross_tab_prop['Not concerned'] #.describe()
cross_tab_prop.sort_values('very-not concerned', ascending=False)

In [ ]:
#make colormap: 
clrs = cm.get_cmap('RdYlBu_r', 22)

plt.figure(figsize=(6, 6))
cross_tab_prop.plot(kind='line', 
                    colormap=clrs, 
                    figsize=(10, 6))
plt.legend(bbox_to_anchor=(1.0, 1.0), ncol=2)
#plt.xlabel("Release Year")
plt.ylabel("Proportion for each Q14.9 response cat")
plt.show()

In [ ]:
cross_tab_prop = pd.crosstab(index=summary_df['Q14.9'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")

cross_tab_prop = cross_tab_prop.reindex(index = ['Not concerned', 'Rather not concerned', 'Neither', 'Quite concerned', 'Very concerned'])

cross_tab = pd.crosstab(index=summary_df['Q14.9'],
                        columns=summary_df['Dominant_Topic'])

cross_tab = cross_tab.reindex(index = ['Not concerned', 'Rather not concerned', 'Neither', 'Quite concerned', 'Very concerned'])
cross_tab

In [ ]:
plt.figure(figsize=(6, 6))
cross_tab_prop.plot(kind='line', 
                    colormap=clrs, 
                    figsize=(10, 6))
plt.legend(bbox_to_anchor=(1.25, 1.00), ncol=2)
plt.xlabel("climate concern")
plt.ylabel("Proportion")
plt.show()

In [ ]:
summary_df = survey_df_w_lda[['Q14.9', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['Q14.9'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")

cross_tab_prop = cross_tab_prop.reindex(index = ['Not concerned', 'Rather not concerned', 'Neither', 'Quite concerned', 'Very concerned'])
cross_tab_prop 

cross_tab= pd.crosstab(index=summary_df['Q14.9'],
                             columns=summary_df['Dominant_Topic'])

cross_tab = cross_tab.reindex(index = ['Not concerned', 'Rather not concerned', 'Neither', 'Quite concerned', 'Very concerned'])
cross_tab

cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap=clrs, 
                    figsize=(8, 8))


for n, x in enumerate([*cross_tab.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop.loc[x],
                                   cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=n - 0.17,
                 y=(y_loc - proportion) + (proportion / 5),
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=10) #,
                 #fontweight="bold")

plt.legend(loc="upper left", ncol=2)
plt.xlabel("Q14.9 How concerned are you about climate change?")
plt.ylabel("Proportion")
plt.legend(title='Dominant LDA Topic', bbox_to_anchor=(1.025, 1.00), ncol=2)
fig_name = 'stacked_bar'
file_name=('../../figures/' + fig_name + ('_ldamallet_12_topic_proportion_across_Q14.9.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

### Acceptance/Rejection of densification projects

In [ ]:
summary_df = survey_df_w_lda[['Q7.5', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['Q7.5'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")

cross_tab_prop = cross_tab_prop.reindex(index = ['always reject', 'reject', 'mostly reject', 'undecided', 'mostly accept', 'accept', 'always accept'])
cross_tab_prop 

In [ ]:
cross_tab= pd.crosstab(index=summary_df['Q7.5'],
                             columns=summary_df['Dominant_Topic'])

cross_tab = cross_tab.reindex(index = ['always reject', 'reject', 'mostly reject', 'undecided', 'mostly accept', 'accept', 'always accept'])
cross_tab

In [ ]:
plt.figure(figsize=(6, 6))
cross_tab_prop.plot(kind='line', 
                    colormap=clrs, 
                    figsize=(10, 6))
plt.legend(bbox_to_anchor=(1.0, 1.0), ncol=2)
plt.show()

In [ ]:
summary_df = survey_df_w_lda[['Q7.5', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['Q7.5'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")

cross_tab_prop = cross_tab_prop.reindex(index = ['always reject', 'reject', 'mostly reject', 'undecided', 'mostly accept', 'accept', 'always accept'])

cross_tab= pd.crosstab(index=summary_df['Q7.5'],
                             columns=summary_df['Dominant_Topic'])

cross_tab = cross_tab.reindex(index = ['always reject', 'reject', 'mostly reject', 'undecided', 'mostly accept', 'accept', 'always accept'])


clrs = cm.get_cmap('RdYlBu_r', 22)

plt.figure(figsize=(6, 6))
cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap=clrs, 
                    figsize=(10, 6))


for n, x in enumerate([*cross_tab.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop.loc[x],
                                   cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=n - 0.17,
                 y=(y_loc - proportion) + (proportion / 5),
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=10) #,
                 #fontweight="bold")

plt.legend(loc="upper left", ncol=2)
plt.xlabel("Q7.5 Do you personally tend to accept or reject projects that aim to increase building density?")
plt.ylabel("Proportion")
plt.legend(title='Dominant LDA Topic', bbox_to_anchor=(1.025, 1.00), ncol=2)
fig_name = 'stacked_bar'
file_name=('../../figures/' + fig_name + ('_ldamallet_12_topic_proportion_across_Q7.5.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

### Frame

In [ ]:
summary_df = survey_df_w_lda[['frame', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['frame'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")

cross_tab_prop = cross_tab_prop.reindex(index = ['neighborhood', 'district', 'other district'])
cross_tab_prop 

cross_tab= pd.crosstab(index=summary_df['frame'],
                             columns=summary_df['Dominant_Topic'])

cross_tab_prop = cross_tab_prop.reindex(index = ['neighborhood', 'district', 'other district'])
cross_tab

cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap=clrs, 
                    figsize=(4, 8))


for n, x in enumerate([*cross_tab.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop.loc[x],
                                   cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=n - 0.17,
                 y=(y_loc - proportion) + (proportion / 3),
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=10) #,
                 #fontweight="bold")

plt.legend(loc="upper left", ncol=2)
plt.xlabel("frame: Project location")
plt.ylabel("Proportion")
plt.legend(title='Dominant LDA Topic', bbox_to_anchor=(1.025, 1.00), ncol=2)
fig_name = 'stacked_bar'
file_name=('../../figures/' + fig_name + ('_ldamallet_12_topic_proportion_across_frame.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

### Political leaning

In [ ]:
summary_df = survey_df_w_lda[['Q14.1_1', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['Q14.1_1'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")


cross_tab= pd.crosstab(index=summary_df['Q14.1_1'],
                             columns=summary_df['Dominant_Topic'])


cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap=clrs, 
                    figsize=(12, 8))


for n, x in enumerate([*cross_tab.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop.loc[x],
                                   cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=n - 0.17,
                 y=(y_loc - proportion) + (proportion / 3),
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=10) #,
                 #fontweight="bold")

plt.legend(loc="upper left", ncol=2)
plt.xlabel("political leaning 0- most left and 7- most right")
plt.ylabel("Proportion")
plt.legend(title='Dominant LDA Topic', bbox_to_anchor=(1.025, 1.00), ncol=2)
fig_name = 'stacked_bar'
file_name=('../../figures/' + fig_name + ('_ldamallet_12_topic_proportion_across_Q14.1_1.png'))
#plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()

### City

In [ ]:
summary_df = survey_df_w_lda[['city', 'Dominant_Topic']] #)#['Dominant_Topic']#.count()
print(len(summary_df))
summary_df.head()

cross_tab_prop = pd.crosstab(index=summary_df['city'],
                             columns=summary_df['Dominant_Topic'],
                             normalize="index")


cross_tab= pd.crosstab(index=summary_df['city'],
                             columns=summary_df['Dominant_Topic'])


cross_tab_prop.plot(kind='bar', 
                    stacked=True, 
                    colormap=clrs, 
                    figsize=(6, 8))


for n, x in enumerate([*cross_tab.index.values]):
    for (proportion, y_loc) in zip(cross_tab_prop.loc[x],
                                   cross_tab_prop.loc[x].cumsum()):
                
        plt.text(x=n - 0.17,
                 y=(y_loc - proportion) + (proportion / 3),
                 s=f'{np.round(proportion * 100, 1)}%', 
                 color="black",
                 fontsize=10) #,
                 #fontweight="bold")

plt.legend(loc="upper left", ncol=2)
plt.xlabel("City")
plt.ylabel("Proportion")
plt.legend(title='Dominant LDA Topic', bbox_to_anchor=(1.025, 1.00), ncol=2)
fig_name = 'stacked_bar'
#file_name=('../../figures/' + fig_name + ('_ldamallet_12_topic_proportion_across_city2.png'))
plt.savefig((file_name), dpi=300, bbox_inches='tight')
plt.show()